Скачать датасет с фразами Симпсонов - github.com...psons-Data

Обучить word2vec на фразах персонажей

Визуализировать embeddings по самым частотным словам (top 1000)

Найти самые близкие слова для: homer - marge + bart bart - lisa + school marge - homer + home

5* Попробовать построить классификатор bart/lisa с использованием этих эмбеддингов

In [ ]:
import pandas as pd
from gensim import models

import torch
from sklearn.model_selection import train_test_split

import time

In [ ]:
!wget https://github.com/sujanjoejacob/Text-mining-with-Simpsons-Data/archive/refs/heads/master.zip
!unzip master.zip
!mv Text-mining-with-Simpsons-Data-master/simpsons_script_lines.csv.zip ./
!unzip simpsons_script_lines.csv.zip

--2023-07-05 09:20:05--  https://github.com/sujanjoejacob/Text-mining-with-Simpsons-Data/archive/refs/heads/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/sujanjoejacob/Text-mining-with-Simpsons-Data/zip/refs/heads/master [following]
--2023-07-05 09:20:05--  https://codeload.github.com/sujanjoejacob/Text-mining-with-Simpsons-Data/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [    <=>             ]   8.94M  9.60MB/s    in 0.9s    

2023-07-05 09:20:06 (9.60 MB/s) - ‘master.zip’ saved [9376659]

Archive:  master.zip
4d1aee02aecdbd0b26e20e6cc2a5ce046c

In [ ]:
df = pd.read_csv('simpsons_script_lines.csv', usecols=['raw_character_text', 'spoken_words', 'normalized_text', 'word_count'])
df.head()

,raw_character_text,spoken_words,normalized_text,word_count
0,Miss Hoover,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,Lisa Simpson,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,Miss Hoover,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,Lisa Simpson,That life is worth living.,that life is worth living,5
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [ ]:
df = df.dropna()
df = df.drop_duplicates()
df.reset_index(inplace=True)

In [ ]:
sentences = [i.split(' ') for i in df['normalized_text'].tolist()]

In [ ]:
model = models.Word2Vec(min_count=2)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(4918440, 6476390)

In [ ]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []

fd = FreqDist()

for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])

print(top_words[:50:])
top_words_vec = model.wv[top_words]

<ipython-input-8-5cb28275eddd>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm(sentences):


  0%|          | 0/126875 [00:00<?, ?it/s]

['the', 'you', 'i', 'a', 'to', 'and', 'of', 'it', 'that', 'my', 'in', 'is', 'this', 'me', 'your', 'for', 'im', 'we', 'on', 'oh', 'have', 'what', 'but', 'be', 'no', 'well', 'its', 'dont', 'all', 'just', 'with', 'do', 'are', 'now', 'not', 'like', 'so', 'was', 'get', 'youre', 'can', 'know', 'one', 'at', 'thats', 'out', 'up', 'here', 'hey', 'if']


In [ ]:
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE top1000 words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [ ]:
print(model.wv.most_similar(positive=["marge", "bart"], negative=["marge"], topn=5))

[('lisa', 0.9148375391960144), ('homer', 0.8836032152175903), ('milhouse', 0.809614896774292), ('maggie', 0.7758014798164368), ('dad', 0.7654369473457336)]


In [ ]:
print(model.wv.most_similar(positive=["bart", "school"], negative=["lisa"], topn=5))

[('game', 0.7339708805084229), ('church', 0.7307146191596985), ('house', 0.7181503176689148), ('christmas', 0.6959559917449951), ('store', 0.6880595088005066)]


In [ ]:
print(model.wv.most_similar(positive=["marge", "home"], negative=["homer"], topn=5))

[('back', 0.7687998414039612), ('sleep', 0.6749753355979919), ('bed', 0.6496912240982056), ('dinner', 0.6429943442344666), ('while', 0.6024989485740662)]


## Bart Lisa classifier

In [ ]:
df = pd.read_csv('simpsons_script_lines.csv', usecols=['raw_character_text', 'normalized_text'])
df.head()

,raw_character_text,normalized_text
0,Miss Hoover,no actually it was a little of both sometimes ...
1,Lisa Simpson,wheres mr bergstrom
2,Miss Hoover,i dont know although id sure like to talk to h...
3,Lisa Simpson,that life is worth living
4,Edna Krabappel-Flanders,the polls will be open from now until the end ...


In [ ]:
df = df.dropna()
df = df.drop_duplicates()
df.reset_index(inplace=True)

In [ ]:
df.loc[df['raw_character_text'].str.contains('Bart Sim'), 'class'] = 0
df.loc[df['raw_character_text'].str.contains('Lisa Sim'), 'class'] = 1

In [ ]:
df.dropna(inplace=True)
df['class'] = df['class'].astype(int)

In [ ]:
df['raw_character_text'].value_counts()

Bart Simpson    11887
Lisa Simpson    10016
Name: raw_character_text, dtype: int64

In [ ]:
df['tokens'] = df['normalized_text'].apply(lambda x: x.split(' '))

In [ ]:
df.head()

,index,raw_character_text,normalized_text,class,tokens
1,1,Lisa Simpson,wheres mr bergstrom,1,"[wheres, mr, bergstrom]"
3,3,Lisa Simpson,that life is worth living,1,"[that, life, is, worth, living]"
7,7,Bart Simpson,victory party under the slide,0,"[victory, party, under, the, slide]"
8,9,Lisa Simpson,mr bergstrom mr bergstrom,1,"[mr, bergstrom, mr, bergstrom]"
10,11,Lisa Simpson,do you know where i could find him,1,"[do, you, know, where, i, could, find, him]"


In [ ]:
MAX_LEN = max([len(i.split(' ')) for i in df['normalized_text']])
vector_size = model.wv.get_vector('bergstrom').shape[0]
train_size = int(len(df['normalized_text']) / 100 * 80)
test_size = len(df['normalized_text']) - int(len(df['normalized_text']) / 100 * 80)
MAX_LEN, train_size, test_size, test_size+train_size == len(df['normalized_text'])

(75, 17522, 4381, True)

In [ ]:
train = []

for ida, sent in enumerate(df['normalized_text'].tolist()[:train_size]):
    X = torch.zeros(MAX_LEN, vector_size)
    for idb, word in enumerate(sent.split(' ')):
      if word in model.wv:
        X[idb] = torch.tensor(model.wv[word])
    train.append((X.T, df['class'].values[ida]))

test = []

for ida, sent in enumerate(df['normalized_text'].tolist()[train_size:]):
    X = torch.zeros(MAX_LEN, vector_size)
    for idb, word in enumerate(sent.split(' ')):
      if word in model.wv:
        X[idb] = torch.tensor(model.wv[word])
    test.append((X.T, df['class'].values[ida + train_size]))

In [ ]:
train_ds = torch.utils.data.DataLoader(train, batch_size=256, shuffle=True)
test_ds = torch.utils.data.DataLoader(train, batch_size=256)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(MAX_LEN * vector_size, 1024),
    torch.nn.BatchNorm1d(1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 2)
)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=7500, out_features=1024, bias=True)
  (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): Dropout(p=0.2, inplace=False)
  (5): Linear(in_features=1024, out_features=512, bias=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=512, out_features=256, bias=True)
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Dropout(p=0.2, inplace=False)
  (13): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
def train_model(epochs, batch_size, model, trainer, loss, train_ds, test_ds):

  for ep in range(epochs):

    train_iter, train_pass = 0, 0
    train_loss, train_acc = 0., 0.

    start = time.time()

    model.train()

    for X, y in train_ds:
      trainer.zero_grad()
      y_pred = model(X)
      l = loss(y_pred, y)
      l.backward()
      trainer.step()
      train_loss += l.item()
      train_acc += (y_pred.argmax(dim=1) == y).sum().item()
      train_iter += 1
      train_pass += len(X)

    test_iter, test_pass  = 0, 0
    test_loss, test_acc = 0., 0.

    model.eval()

    for X, y in test_ds:
      y_pred = model(X)
      l = loss(y_pred, y)
      test_loss += l.item()
      test_acc += (y_pred.argmax(dim=1) == y).sum().item()
      test_iter += 1
      test_pass += len(X)

    print(f"epoch: {ep}",
          f"time: {time.time() - start}",
          f"train_loss: {train_loss / train_iter}",
          f"train_acc: {train_acc / train_pass}",
          f"test_loss: {test_loss / test_iter}",
          f"test_acc: {test_acc / test_pass}")

In [ ]:
train_model(epochs=10, batch_size=256, model=model, trainer=trainer, loss=loss, train_ds=train_ds, test_ds=test_ds)

epoch: 0 time: 20.053135871887207 train_loss: 0.7015337209770645 train_acc: 0.5487957995662596 test_loss: 0.649092770141104 test_acc: 0.6262412966556329
epoch: 1 time: 19.11611580848694 train_loss: 0.6488060502038486 train_acc: 0.6161967811893619 test_loss: 0.5908160814340564 test_acc: 0.6925579271772628
epoch: 2 time: 17.891772747039795 train_loss: 0.6085349105406499 train_acc: 0.6563748430544458 test_loss: 0.5501939865989961 test_acc: 0.7185252825019975
epoch: 3 time: 19.21077251434326 train_loss: 0.5611412823200226 train_acc: 0.6923867138454515 test_loss: 0.5082496560138204 test_acc: 0.7339915534756306
epoch: 4 time: 18.262322902679443 train_loss: 0.5207094766091609 train_acc: 0.7209793402579614 test_loss: 0.4451353973236637 test_acc: 0.7739984020089031
epoch: 5 time: 19.759217262268066 train_loss: 0.4689821838468745 train_acc: 0.7521972377582468 test_loss: 0.3985378962496053 test_acc: 0.8056157972834151
epoch: 6 time: 18.78598976135254 train_loss: 0.42636616204095923 train_acc: 0.7